Connected to .venv (Python 3.10.11)

In [1]:
from insightface.app import FaceAnalysis
import cv2
import numpy as np


In [4]:
import shutil, pickle, os

EMB_PATH = "embeddings_arcface.pkl"
user_name = "Manish"  # delete this user

# delete images
folder_path = f"enrolled_images/{user_name}"
shutil.rmtree(folder_path, ignore_errors=True)

# delete from DB
if os.path.exists(EMB_PATH):
    with open(EMB_PATH, 'rb') as f:
        db = pickle.load(f)
    if user_name in db:
        del db[user_name]
        with open(EMB_PATH, 'wb') as f:
            pickle.dump(db, f)
        print(f"Deleted {user_name} images + embedding.")
    else:
        print("User not found in DB.")
else:
    print("DB file not found.")


Deleted Manish images + embedding.


TESTING -3

In [8]:
import os
import cv2
import pickle
import numpy as np
from insightface.app import FaceAnalysis
import time

# ----------------- Config -----------------
EMB_PATH = "embeddings_arcface.pkl"

# ----------------- Initialize ArcFace -----------------
device = 0  # 0 = GPU, -1 = CPU
arcface_app = FaceAnalysis(
    name='buffalo_l',
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
)
arcface_app.prepare(ctx_id=device, det_size=(640, 640))

# ----------------- Load or Create DB -----------------
if os.path.exists(EMB_PATH):
    with open(EMB_PATH, 'rb') as f:
        embeddings_db = pickle.load(f)
    print("Loaded embeddings DB:", list(embeddings_db.keys()))
else:
    embeddings_db = {}
    print("Starting with empty DB")

# ----------------- Liveliness (Blink) Check -----------------
def is_blink_detected(face_landmarks, threshold=0.2):
    """
    Very simple blink check using eye position difference.
    InsightFace returns 5 landmarks:
    0: left eye, 1: right eye, 2: nose tip, 3: left mouth, 4: right mouth
    """
    if face_landmarks is None or len(face_landmarks) < 5:
        return False
    left_eye = face_landmarks[0]
    right_eye = face_landmarks[1]
    # Use y-difference between eyes as a proxy
    eye_dist = abs(left_eye[1] - right_eye[1])
    return eye_dist < threshold  # crude check

# ----------------- Enrollment -----------------
def enroll_user_from_camera(name, num_images=50, wait_between=0.3):
    cap = cv2.VideoCapture(0)
    captured = 0
    embeds = []

    save_folder = f"enrolled_images/{name}"
    os.makedirs(save_folder, exist_ok=True)
    print(f"Starting enrollment for {name}")

    while captured < num_images:
        ret, frame = cap.read()
        if not ret:
            break

        faces = arcface_app.get(frame)
        if faces:
            face = faces[0]
            emb = face.embedding

            # crude blink/liveliness check
            if face.kps is not None:
                blink = is_blink_detected(face.kps)
                if blink:
                    print("Blink detected ✅")

            embeds.append(emb)

            # clamp bbox and save cropped face image safely
            x1, y1, x2, y2 = face.bbox.astype(int)
            h, w = frame.shape[:2]
            x1 = max(0, x1); y1 = max(0, y1)
            x2 = min(w, x2); y2 = min(h, y2)

            if x2 > x1 and y2 > y1:
                face_img = frame[y1:y2, x1:x2]
                if face_img.size != 0:
                    cv2.imwrite(f"{save_folder}/{captured+1}.jpg", face_img)

            # Draw rectangle
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{captured+1}/{num_images}", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            captured += 1
            time.sleep(wait_between)

        cv2.imshow("Enrollment", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if embeds:
        mean_emb = np.mean(np.array(embeds), axis=0)
        mean_emb /= np.linalg.norm(mean_emb)
        embeddings_db[name] = mean_emb
        with open(EMB_PATH, 'wb') as f:
            pickle.dump(embeddings_db, f)
        print(f"Enrollment complete for {name}, images saved at {save_folder}")
    else:
        print("Enrollment failed")

# ----------------- Main Loop -----------------
while True:
    person_name = input("Enter the name of the person to enroll: ")
    enroll_user_from_camera(person_name, num_images=200, wait_between=0.3)
    more = input("Do you want to enroll another person? (y/n): ")
    if more.lower() != 'y':
        print("Enrollment finished for all users.")
        break  # <— only break, removed the comment text


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

In [9]:
import cv2
import time
import numpy as np
from collections import deque
import mediapipe as mp
from insightface.app import FaceAnalysis
import pickle
import psutil
import GPUtil

# ---------------- CONFIG ----------------
EMB_PATH = "embeddings_arcface.pkl"
RECOG_THRESHOLD = 0.55
EAR_CONSEC_FRAMES = 2
BLINK_WINDOW_SECONDS = 5
MOVEMENT_THRESHOLD = 2.0
PROCESS_FACE_MESH_EVERY = 5  # run face mesh for liveness every 5 frames

# ---------------- GPU Setup ----------------
device = 0  # GPU
arcface_app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
arcface_app.prepare(ctx_id=device, det_size=(320,320))

# ---------------- Face Mesh Setup ----------------
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=3,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

def eye_aspect_ratio(landmarks, eye_indices, w, h):
    coords = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_indices]
    A = np.linalg.norm(np.array(coords[1])-np.array(coords[5]))
    B = np.linalg.norm(np.array(coords[2])-np.array(coords[4]))
    C = np.linalg.norm(np.array(coords[0])-np.array(coords[3]))
    return (A+B)/(2.0*C)

# ---------------- Load embeddings ----------------
with open(EMB_PATH, "rb") as f:
    embeddings_db = pickle.load(f)

def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def identify_face(emb):
    emb = emb/np.linalg.norm(emb)
    best_score,best_name = 0,"Unknown"
    for name,ref_emb in embeddings_db.items():
        ref_emb = ref_emb/np.linalg.norm(ref_emb)
        score = cosine_similarity(emb,ref_emb)
        if score > best_score:
            best_score,best_name = score,name
    if best_score < RECOG_THRESHOLD:
        best_name = "Unknown"
    return best_name,best_score

# ---------------- Live Recognition ----------------
def live_webcam_recognition():
    cap = cv2.VideoCapture(0)
    face_tracks = {}
    prev_time = time.time()
    frame_count = 0
    gpu = GPUtil.getGPUs()[0]
    print(f"✅ Using GPU: {gpu.name}, Total Memory: {gpu.memoryTotal} MB")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        start_time = time.time()
        h, w = frame.shape[:2]

        # Resize for speed
        small_frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)

        # ---------------- ArcFace ----------------
        faces = arcface_app.get(small_frame)

        # ---------------- Face Mesh every N frames ----------------
        landmarks_all = []
        if frame_count % PROCESS_FACE_MESH_EVERY == 0:
            rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb)
            landmarks_all = results.multi_face_landmarks if results.multi_face_landmarks else []

        for idx, face in enumerate(faces):
            # Scale bbox back to original size
            x1,y1,x2,y2 = (face.bbox.astype(int) * 2)
            emb = face.embedding
            name,score = identify_face(emb)
            color = (0,255,0) if name!="Unknown" else (0,0,255)
            face_id = f"{name}_{idx}"

            if face_id not in face_tracks:
                face_tracks[face_id] = {
                    "blink_times": deque(),
                    "ear_counter":0,
                    "ear_baseline":0.26,
                    "last_pos":((x1+x2)//2,(y1+y2)//2),
                    "movement":0.0
                }

            is_live, ear_val = False,None
            if landmarks_all:
                best_mesh = min(landmarks_all, key=lambda lm: abs((lm.landmark[1].x*w/2)-(x1+x2)/4))
                lm = best_mesh.landmark
                leftEAR = eye_aspect_ratio(lm,LEFT_EYE,w/2,h/2)
                rightEAR = eye_aspect_ratio(lm,RIGHT_EYE,w/2,h/2)
                ear_val = (leftEAR+rightEAR)/2.0

                base = face_tracks[face_id]["ear_baseline"]
                ear_threshold = base*0.8

                if ear_val<ear_threshold:
                    face_tracks[face_id]["ear_counter"]+=1
                else:
                    if face_tracks[face_id]["ear_counter"]>=EAR_CONSEC_FRAMES:
                        face_tracks[face_id]["blink_times"].append(time.time())
                    face_tracks[face_id]["ear_counter"]=0
                    face_tracks[face_id]["ear_baseline"]=0.9*base+0.1*ear_val

                now = time.time()
                while face_tracks[face_id]["blink_times"] and (now-face_tracks[face_id]["blink_times"][0])>BLINK_WINDOW_SECONDS:
                    face_tracks[face_id]["blink_times"].popleft()

                blink_count = len(face_tracks[face_id]["blink_times"])
                is_live = blink_count>0

                # Movement
                cx,cy = (x1+x2)//2,(y1+y2)//2
                lx,ly = face_tracks[face_id]["last_pos"]
                move_dist = np.hypot(cx-lx, cy-ly)
                face_tracks[face_id]["movement"]=0.8*face_tracks[face_id]["movement"]+0.2*move_dist
                face_tracks[face_id]["last_pos"]=(cx,cy)
                is_live = is_live or (face_tracks[face_id]["movement"]>MOVEMENT_THRESHOLD)

            # Draw
            cv2.rectangle(frame,(x1,y1),(x2,y2),color,2)
            cv2.putText(frame,f"{name} ({score*100:.0f}%)",(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,color,2)
            status_text="LIVE" if is_live else "NOT LIVE"
            status_color=(0,255,0) if is_live else (0,0,255)
            cv2.putText(frame,f"Liveness: {status_text}",(x1,y2+25),cv2.FONT_HERSHEY_SIMPLEX,0.7,status_color,2)
            if ear_val:
                cv2.putText(frame,f"EAR: {ear_val:.3f}",(x1,y2+45),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,0),2)
            cv2.putText(frame,f"Movement: {face_tracks[face_id]['movement']:.1f}",(x1,y2+65),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)

        # FPS
        curr_time = time.time()
        fps = 1/(curr_time-prev_time)
        prev_time = curr_time
        inf_time = (time.time()-start_time)*1000
        mem_usage = psutil.virtual_memory().percent
        gpu_load = gpu.load*100
        gpu_mem = gpu.memoryUsed

        cv2.putText(frame,f"FPS: {fps:.1f}",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)
        cv2.putText(frame,f"Inference: {inf_time:.1f}ms",(10,60),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)
        cv2.putText(frame,f"Memory: {mem_usage:.1f}%",(10,90),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)
        cv2.putText(frame,f"GPU: {gpu_load:.1f}% | VRAM: {gpu_mem:.1f}MB",(10,120),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)

        cv2.imshow("GPU Face Recognition + Liveness",frame)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# ---------------- Run ----------------
live_webcam_recognition()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

COMPRESSION

In [7]:
import onnxruntime as ort

print("Available providers:", ort.get_available_providers())


Available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [9]:
from insightface.app import FaceAnalysis

# 1️⃣ GPU first, CPU fallback
arcface_app = FaceAnalysis(
    name='buffalo_l',
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
)

# 2️⃣ ctx_id=0 = first GPU
arcface_app.prepare(ctx_id=0, det_size=(640, 640))

# 3️⃣ आता model GPU वर चालेल
print("✅ FaceAnalysis ready on GPU!")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ganes/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

In [10]:
import onnxruntime as ort
print(ort.get_available_providers())


['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
